In [14]:
# Import general python packages used by scientists
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import gc
import os
import pandas as pd

# Import packages  Image Access
import lsst.daf.butler as dafButler
import lsst.geom as geom
from lsst.geom import PointD
from lsst.geom import Point2D
import lsst.afw.display as afwDisplay
import lsst.daf.base as dafBase
from lsst.daf.butler import Butler
import lsst.afw.image as afwImage
import lsst.afw.table as afwTable
from lsst.afw.geom.ellipses import Quadrupole, SeparableDistortionTraceRadius
from lsst.afw import cameraGeom

# Import packages for  Catalog Access
import pandas
pandas.set_option('display.max_rows', 1000)
from lsst.rsp import get_tap_service, retrieve_query

#Import custom packages
from ellipticity_mapping import calculate_ellipticity_on_xy
from collection_dictionary_shared import collection_dictionary

from datetime import datetime, timedelta
from astroplan import Observer
from astropy.coordinates import EarthLocation
from astropy.time import Time
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
from pytz import all_timezones #To visualize all the possible timezones
from pytz import timezone
import datetime
import time

In [15]:
def remove_figure(fig):
    """
    Remove a figure to reduce memory footprint.

    Parameters
    ----------
    fig: matplotlib.figure.Figure
        Figure to be removed.

    Returns
    -------
    None
    """
    # get the axes and clear their images
    for ax in fig.get_axes():
        for im in ax.get_images():
            im.remove()
    fig.clf()       # clear the figure
    plt.close(fig)  # close the figure
    gc.collect()    # call the garbage collector
    
def pixel_to_focal(x, y, det):
    """
    Parameters
    ----------
    x, y : array
        Pixel coordinates.
    det : lsst.afw.cameraGeom.Detector
        Detector of interest.
    Returns
    -------
    fpx, fpy : array
        Focal plane position in millimeters in DVCS
        See https://lse-349.lsst.io/
    """
    tx = det.getTransform(cameraGeom.PIXELS, cameraGeom.FOCAL_PLANE)
    fpx, fpy = tx.getMapping().applyForward(np.vstack((x, y)))
    return fpx.ravel(), fpy.ravel()

def pixel_to_focal_angle(x, y, det):
    """
    Parameters
    ----------
    x, y : array
        Pixel coordinates.
    det : lsst.afw.cameraGeom.Detector
        Detector of interest.
    Returns
    -------
    fpx, fpy : array
        Focal plane position in degrees in DVCS
        See https://lse-349.lsst.io/
    """
    tx = det.getTransform(cameraGeom.PIXELS, cameraGeom.FIELD_ANGLE)
    fpx, fpy = tx.getMapping().applyForward(np.vstack((x, y)))
    return np.degrees(fpx.ravel()), np.degrees(fpy.ravel())

In [16]:
print("potrei leggere queste cose da yaml???")
seqnums = [36, 37, 38, 39, 40, 41, 42, 43, 44]
detectors = [168, 178, 188, 123, 132, 142, 152, 161, 75, 84, 94, 104, 113, 27, 36, 46, 56, 65, 0, 10, 20]
# seqnums = [53]
# detectors = list(np.arange(189))
seqnums = [61]
detectors = [168, 178, 84, 94, 10, 20]
format_figures = 'png'

potrei leggere queste cose da yaml???


In [17]:
collection_dict = collection_dictionary()
collection_dict_seqnum61_gains = {61: 'pipetask_output/20240311T104052Z'}
collection_dict_seqnum61_nogains = {61: 'pipetask_output/20240311T104944Z'}

In [18]:
folder = '/sdf/data/rubin/shared/image_quality/imsim/'
#Define the butler data configuration and collection (una tantum )
config = folder+'repo'
folderout = folder+'ellipticitymap/'
subfolderout_fig = 'figures/'
subfolderout_tab = 'tables/'

In [20]:
for seqnum in seqnums:

    fpxs = []
    fpys = []
    mags = []
    
    for detector in detectors:
        
        visit_seqnum = 5023071800000 + seqnum        
        collection = collection_dict_seqnum61_nogains[seqnum]

        # Create the butler
        butler = dafButler.Butler(config,collections=collection)

        #Adesso dobbiammo dire al butler che tipo di dati vogliamo.
        #La call si fa chiedendo un datasetType (e.g., deepCoadd, calexp, objectTable) e un data ID(is a dictionary-like identifier for a specific data product)
        #Qui piu' informazioni sul butler 
        #https://github.com/rubin-dp0/tutorial-notebooks/blob/main/04b_Intermediate_Butler_Queries.ipynb

        datasetType='calexp'
        dataId = {'visit': visit_seqnum, 'detector': detector, 'band':'r'}
        calexp = butler.get(datasetType, **dataId)
        sources = butler.get('src', dataId)
        ccd = calexp.detector.getId()
        det = calexp.getDetector()
        wcs = calexp.getWcs()
        calexp_info = calexp.getInfo()
        
        xx = [l.getCentroid()[0] for l in sources]
        yy = [l.getCentroid()[1] for l in sources]
        mag = [l['base_PsfFlux_instFlux'] for l in sources]

        fpx = []
        fpy = []
        for xxx, yyy in zip(xx, yy):
            fpx_temp, fpy_temp = pixel_to_focal_angle(xxx, yyy, det)
            fpx.append(fpx_temp[0])
            fpy.append(fpy_temp[0])

        fpxs.append(fpx)
        fpys.append(fpy)
        mags.append(mag)

    mags_final = [i for sublist in mags for i in sublist]
    fpxs_final = [i for sublist in fpys for i in sublist] # INVERSIONE XY
    fpys_final = [i for sublist in fpxs for i in sublist] # INVERSIONE XY

    fig = plt.figure(figsize=(18, 18))
    plt.scatter(fpxs_final, fpys_final, c=mags_final, s=1)
    plt.colorbar()
    plt.xlabel('x [deg]')
    plt.ylabel('y [deg]')
    plt.title('PSF counts')
    fig.savefig(folderout+"figures/PSF_counts_seqnum{:04d}_nogain.{:s}".format(seqnum,format_figures))
    remove_figure(fig)